In [44]:
import numpy as np
import pandas as pd
from scipy import stats

In [16]:
# Represent probabilities of a tier hit
level1 = [1.0, 0, 0, 0, 0]
level2 = [1.0, 0, 0, 0, 0]
level3 = [.65, .3, .5, 0, 0]
level4 = [.5, .35, .15, 0, 0]
level5 = [.37, .35, .25, .03, 0]
level6 = [.245, .35, .3, .10, .005]
level7 = [.2, .3, .33, .15, .02]
level8 = [.15, .25, .35, .2, .05]
level9 = [.1, .15, .35, .3, .1]
levels_df = pd.DataFrame([level1,level2,level3,level4,level5,level6,level7,level8,level9],
                      columns=['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5'])

# Represent tier numbers & frequencies
tier1 = (12, 39)
tier2 = (12, 26)
tier3 = (12, 21)
tier4 = (9, 13)
tier5 = (6, 10)
tiers_df = pd.DataFrame([tier1,tier2,tier3,tier4,tier5], columns=['Unique Characters', 'Characters in Pool'])

In [22]:
tiers_df

,Unique Characters,Characters in Pool
0,12,39
1,12,26
2,12,21
3,9,13
4,6,10


In [47]:
stats.binom.stats(6,.13, moments='mvsk')

(array(0.78), array(0.6786), array(0.89830702), array(0.47362216))

In [48]:
stats.binom.stats(6,.13)

(array(0.78), array(0.6786))

In [31]:
def hit_prob(level, tier, levels_df=levels_df, tiers_df=tiers_df):
    tier_prob = levels_df['Tier {}'.format(tier)][level-1]
    hits_in_pool = tiers_df['Characters in Pool'][tier-1]
    chars_in_tier = tiers_df['Unique Characters'][tier-1]
    hit_prob_solo = (hits_in_pool/(chars_in_tier*hits_in_pool))*tier_prob
    hit_prob = hit_prob_solo*5 # could adjust this to account for dependent probs of sampling
    return hit_prob

#TODO: clean up & account for dependent probs
def multi_hits_prob(level, tiers, levels_df=levels_df, tiers_df=tiers_df):
    multi_hits_prob = 0
    for tier in tiers:
        multi_hits_prob += hit_prob(level, tier, levels_df, tiers_df)
    return multi_hits_prob

def multi_rolls_prob(rolls, level, tiers, levels_df=levels_df, tiers_df=tiers_df):
    
    

# Calculate the probability of a hit of one unit at a given level

In [49]:
level=5
tier=3
print(hit_prob(level,tier))

0.10416666666666666


# Calculate the probability of a hit of multiple units at a given level

In [50]:
level=5
tiers=[3,2,2]
print(multi_hits_prob(level, tiers))

0.39583333333333326


# Calculate the mean & variance of hits given a certain number of rolls

In [51]:
level=5
tiers=[3,2,2]
rolls=6
one_roll_prob=multi_hits_prob(level,tiers)
mean, var, skew, kurt = stats.binom.stats(rolls, one_roll_prob, moments='mvsk')
print('mean = {} \n var = {}'.format(mean,var))

mean = 2.3749999999999996 
 var = 1.4348958333333333
